In [2]:
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
import plotly.figure_factory as ff
from sklearn.decomposition import LatentDirichletAllocation

import pyLDAvis
import pyLDAvis.sklearn

from gensim.models import Word2Vec

In [3]:
df = pd.read_csv('data/comment_on_plan.csv', encoding = "ISO-8859-1")
tokens = nlp(''.join(str(df.Comment.tolist())))

NER

In [4]:
items = [x.text for x in tokens.ents]
Counter(items).most_common(20)

[('Trudeau', 205),
 ('Canada', 149),
 ('Canadians', 93),
 ('first', 61),
 ('China', 50),
 ('US', 44),
 ('Health Canada', 38),
 ('Canadian', 31),
 ('UK', 31),
 ('Moderna', 31),
 ('Justin', 30),
 ('Chinese', 29),
 ('one', 23),
 ('Trump', 22),
 ("O'Toole", 21),
 ('Covid', 20),
 ('today', 18),
 ('LPC', 18),
 ('Liberal', 18),
 ('Conservatives', 15)]

In [5]:
person_list = []
for ent in tokens.ents:
    if ent.label_ == 'PERSON':
        person_list.append(ent.text)
        
person_counts = Counter(person_list).most_common(20)
df_person = pd.DataFrame(person_counts, columns =['text', 'count'])

In [6]:
df_person.head(10)

text  count
0  Trudeau    196
1    Covid     16
2  O'Toole     15
3  Moderna     13
4     Cons     12
5      Con      7
6    Anand      5
7     Govt      5
8      Lib      4
9      Tam      4

In [7]:
norp_list = []
for ent in tokens.ents:
    if ent.label_ == 'NORP':
        norp_list.append(ent.text)
        
norp_counts = Counter(norp_list).most_common(20)
df_norp = pd.DataFrame(norp_counts, columns =['text', 'count'])

In [8]:
df_norp.head(10)

text  count
0      Canadians     93
1       Canadian     31
2        Chinese     29
3  Conservatives     15
4      Americans      6
5        Russian      3
6   Conservative      3
7          Brits      3
8       Pandemic      3
9        British      3

In [9]:
org_list = []
for ent in tokens.ents:
    if ent.label_ == 'ORG':
        org_list.append(ent.text)
        
org_counts = Counter(org_list).most_common(20)
df_org = pd.DataFrame(org_counts, columns =['text', 'count'])

In [10]:
df_org.head(10)

text  count
0  Health Canada     38
1         Justin     26
2          Trump     20
3            LPC     18
4        Liberal     18
5        Moderna     18
6            CBC     14
7             JT     12
8           Libs     11
9         Pfizer     11

In [11]:
gpe_list = []
for ent in tokens.ents:
    if ent.label_ == 'GPE':
        gpe_list.append(ent.text)
        
gpe_counts = Counter(gpe_list).most_common(20)
df_gpe = pd.DataFrame(gpe_counts, columns =['text', 'count'])

In [12]:
df_gpe.head(10)

text  count
0     Canada    149
1      China     50
2         US     44
3         UK     31
4        USA     13
5     Ottawa     11
6    Germany      8
7  Australia      6
8    Alberta      5
9     Russia      5

China got a lot of attention in the comments, they were referring a [failed Canada's multimillion-dollar COVID vaccine deal with China](https://globalnews.ca/news/7302194/canada-coronavirus-vaccine-deal-china/)

In [13]:
df['polarity'] = df['Comment'].map(lambda text: TextBlob(text).sentiment.polarity)
df['comment_len'] = df['Comment'].astype(str).apply(len)
df['word_count'] = df['Comment'].apply(lambda x: len(str(x).split()))

In [14]:
print('5 random comments with the highest positive sentiment polarity: \n')
cl = df.loc[df.polarity == 1, ['Comment']].sample(5).values
for c in cl:
    print(c[0])

5 random comments with the highest positive sentiment polarity: 

JT doing an excellent job.
the best defense is to stay home, wear masks if you must go out, and keep 6 feet apart....can you promote those guidelines..
prorogued parliament is the best they can do
what's the rush? Take your Time, Wait until China has mastered the Vaccine - then we can millions and millions of doses from them to help their economy ...Why?....Because they are Awesome and deserve it
He is PM because he is the best choice to protect Canadians from suffering the pain and shame of a Conservative government.


In [15]:
print('5 random comments with the most neutral sentiment(zero) polarity: \n')
cl = df.loc[df.polarity == 0, ['Comment']].sample(5).values
for c in cl:
    print(c[0])

5 random comments with the most neutral sentiment(zero) polarity: 

One of the countries the CPC was concerned would get vaccines before us has been running a phase 3 trial of that CanSino vaccine O'Toole is so against.
I don't believe a word this guy says.....he has proven it time and time and time.....again! Incompetence, as quickly as possible
Ill wait a couple of years at a minimum before taking it
Maybe they can beg for some doses to get our front line healthcare workers vaccinated this month.
As quickly as possible is not an answer.


In [16]:
df.polarity.min()

-1.0

In [17]:
df.loc[df.polarity <= -0.7]

Comment  polarity  \
97    Too bad the MSM prints snippets of O'Toole's y...     -0.75   
387   I don't know about you guys but my worst fear ...     -1.00   
620   An awful lot of faith is being given to an unp...     -0.75   
1007  Trudeau destroyed the relationship and your bl...     -0.70   
1109  The con base will vote Con every election when...     -0.80   

      comment_len  word_count  
97            152          28  
387           141          24  
620           117          22  
1007           66          10  
1109           76          14

In [18]:
print('5 comments with the most negative polarity: \n')
cl = df.loc[df.polarity <= -0.7, ['Comment']].sample(5).values
for c in cl:
    print(c[0])

5 comments with the most negative polarity: 

Trudeau destroyed the relationship and your blame game is pathetic
An awful lot of faith is being given to an unproven vaccine. What if giving this to our frontline workers goes wrong?
I don't know about you guys but my worst fear is the Canadian government brainwashing and poisoning us by giving us rushed COVID-19 vaccines.
Too bad the MSM prints snippets of O'Toole's yapping. I guess they have to do it for the sake of the elusive 'balance', but it's really, really annoying
The con base will vote Con every election when these sorts of things happen.


As you can see, the conversations became political quickly.

In [20]:
def preprocessor(text):
    
    text = re.sub('<[^>]*>', '', text)
    text = re.sub(r'[^\w\s]','', text)
    text = text.lower()
    return text
    
df['Comment'] = df['Comment'].apply(preprocessor)

In [20]:
df.polarity.describe()

count    1362.000000
mean        0.073778
std         0.232625
min        -1.000000
25%         0.000000
50%         0.000000
75%         0.178571
max         1.000000
Name: polarity, dtype: float64

In [19]:
fig = ff.create_distplot([df['polarity'].tolist()], group_labels=['Comment Polarity Distribution plot'])
fig.show()

In [21]:
fig = ff.create_distplot([df['word_count'].tolist()], group_labels=['Word count Distribution plot'])
fig.show()

Most comments were short, less than 30 words, few comments can get as long as over 200 words.

In [22]:
nlp = spacy.load('en', disable=['ner', 'parser']) 

def lemmatizer(text):        
    sent = []
    doc = nlp(text)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)

df["comment_lemmatize"] =  df.apply(lambda x: lemmatizer(x['Comment']), axis=1)
df['comment_lemmatize'] = df['comment_lemmatize'].str.replace('-PRON-', '')

In [23]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(df['comment_lemmatize'], 20)
df1 = pd.DataFrame(common_words, columns = ['unigram' , 'count'])

fig = px.bar(df1, x='unigram', y='count')
fig.show()

In [24]:
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(df['Comment'], 10)
df2 = pd.DataFrame(common_words, columns = ['bigram' , 'count'])

fig = px.bar(df2, x='bigram', y='count')
fig.show()

a large number of people imitated Trudeau's response on vaccine roll out plan - "as quickly as possible", and it seemed they were skeptical on whatever "Trudeau says". This in align with a recent poll, finds that [Most Canadians believe government is 'making it up as they go' with COVID vaccine](https://nationalpost.com/news/canada/most-canadians-believe-government-is-making-it-up-as-they-go-with-covid-poll-finds).

In [33]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=3,                        # minimum required occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             max_features=5000,             # max number of unique words. Build a vocabulary that only consider the top max_features ordered by term frequency across the corpus
                            )

data_vectorized = vectorizer.fit_transform(df['comment_lemmatize'])

lda_model = LatentDirichletAllocation(n_components=15, # Number of topics
                                      learning_method='online',
                                      random_state=0,       
                                      n_jobs = -1  # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

In [34]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')

/opt/tljh/user/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
5      114.344879  114.308296       1        1  18.107292
6      105.821045  -93.749626       2        1  13.171526
8     -138.334473   95.320763       3        1  11.708954
1       -7.436697  111.259697       4        1   9.585077
11    -207.550629 -105.472603       5        1   7.793837
0       52.817650   14.603765       6        1   6.662111
3      168.010849    6.806950       7        1   5.615738
10     -94.169601 -157.789551       8        1   5.487232
14    -122.849174  -39.059418       9        1   4.355523
12     -94.894264  195.408295      10        1   3.828402
9       24.927401 -180.464874      11        1   3.297786
2      -53.413467   26.832108      12        1   3.277347
13     -12.255850  -72.093201      13        1   2.714887
7     -224.217957   28.070581      14        1   2.209550
4       36.154255  209.427582      15        1   2.184737, topic_info=     Category        Freq      Term       Total  loglift  logprob
727   Default  219.000000  possible  219.000000  30.0000  30.0000
781   Default  211.000000   quickly  211.000000  29.0000  29.0000
1010  Default  262.000000   trudeau  262.000000  28.0000  28.0000
858   Default  205.000000       say  205.000000  27.0000  27.0000
553   Default   51.000000      line   51.000000  26.0000  26.0000
...       ...         ...       ...         ...      ...      ...
556   Topic15    3.370471    little   15.384271   2.3054  -4.4400
466   Topic15    3.263594      hold   16.794015   2.1855  -4.4723
1031  Topic15    2.473647       usa   14.680333   2.0429  -4.7494
137   Topic15    5.092951    canada  198.073271   0.1629  -4.0272
570   Topic15    2.343300       lpc   17.004069   1.8418  -4.8035

[753 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0         1  0.811119        000
0         3  0.081112        000
2         1  0.594174        1st
2        14  0.198058        1st
3        11  0.872058       2019
...     ...       ...        ...
1091      6  0.608448        yes
1091     10  0.060845        yes
1092      9  0.849200  yesterday
1093      1  0.280387       zero
1093      9  0.560774       zero

[1198 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 7, 9, 2, 12, 1, 4, 11, 15, 13, 10, 3, 14, 8, 5])